In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
from tqdm import tqdm_notebook
from string import punctuation
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv', index_col=0)
train.head()

,title,description,price,category_id
item_id,,,,
0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [3]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,title,description,price
item_id,,,
489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0


In [8]:
tqdm_notebook().pandas()

Определим функцию для изменения данных

In [9]:
def ChangeData(data, func):
    data['title'] = data['title'].progress_apply(lambda x : func(x))
    data['description'] = data['description'].progress_apply(lambda x : func(x))
    return

Определим stopwords и знаки препинания, которые будут удалены

In [5]:
nltk.download("stopwords")
specSymb = punctuation + '«»—“”-№–’'
pattern = re.compile('[' + re.escape(specSymb) + ']')
stopwords_ru = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ravil2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def DelStopWords(text):
    text = text.lower()
    new_text = []
    words = text.split(sep=' ')
    for word in words:
        if (word not in stopwords_ru):
            new_text.append(word)
    new_text = " ".join(new_text)
    return new_text

Удаление stopwords

In [14]:
ChangeData(train, DelStopWords)
ChangeData(test, DelStopWords)

Удаление знаков препинания

In [15]:
ChangeData(train, lambda x : re.sub(pattern, ' ', x))
ChangeData(test, lambda x : re.sub(pattern, ' ', x))

In [16]:
stemmer = SnowballStemmer("russian").stem

In [17]:
def stemming(text):
    words = text.split(sep=' ')
    new_text = []
    for word in words:
        new_text.append(stemmer(word))
    res = ' '.join(new_text)
    return res

Приведем все слова к нормальной форме, для этого воспользуемся SnowballStammer от библиотеки nltk

In [18]:
ChangeData(train, stemming)
ChangeData(test, stemming)

In [19]:
def DelOneSymbol(text):
    res = []
    words = text.split(sep=' ')
    for word in words:
        if len(word) > 1:
            res.append(word)
    res = ' '.join(res)
    return res

Некоторые слова сократились до 1 символа, так как один символ будет скорее зашумлять текст, чем давать полезную информацию, удалим все односимвольные слова:

In [20]:
ChangeData(train, DelOneSymbol)
ChangeData(test, DelOneSymbol)

In [21]:
train.to_csv('new_train.csv')
test.to_csv('new_test.csv')